## 이번 코드를 돌리려면 4장에 00_custum_bundle을 먼저 돌려서 Data를 ingest해주세요.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os 
import pandas as pd 
from pathlib import Path

## Ticker 가져오기

In [3]:
DATA_STORE = Path('../data/', 'assets.h5')

In [4]:
with pd.HDFStore(DATA_STORE) as store:
    stocks = store['finance_datareader/stocks']

In [5]:
stocks

,name
code,
095570,AJ네트웍스
006840,AK홀딩스
027410,BGF
282330,BGF리테일
138930,BNK금융지주
...,...
079980,휴비스
005010,휴스틸
069260,휴켐스


## 먼저 zipline ingest되어 있는 폴더 위치를 환경변수로 넘겨 줘야 된다.

In [6]:
# 이 파일 경로는 컴퓨터마다 OS 마다 다르다.
os.environ['ZIPLINE_ROOT'] = 'C:/Users/PC/.zipline/'

In [7]:
from zipline.utils.run_algo import load_extensions

In [8]:
load_extensions(default=True,
                extensions=[],
                strict=True,
                environ=None)

## 데이터 가져오기

In [9]:
from zipline.data import bundles

bundle_data = bundles.load('finance_datareader')

In [10]:
# zipline에 들어가 있는 ticker 데이터들만 가져오기
tickers = stocks.index
assets = []
for ticker in tickers:
    try:
        asset = bundle_data.asset_finder.lookup_symbol(ticker, as_of_date=None)
        assets.append(asset)
    except Exception as e:
        print(e) 

Symbol '011150' was not found.
Symbol '375500' was not found.
Symbol '383220' was not found.
Symbol '383800' was not found.
Symbol '008260' was not found.
Symbol '302440' was not found.
Symbol '361610' was not found.
Symbol '000270' was not found.
Symbol '001260' was not found.
Symbol '068270' was not found.
Symbol '248070' was not found.
Symbol '002700' was not found.
Symbol '015540' was not found.
Symbol '001780' was not found.
Symbol '380440' was not found.
Symbol '049800' was not found.
Symbol '101140' was not found.
Symbol '013890' was not found.
Symbol '002780' was not found.
Symbol '950210' was not found.
Symbol '000080' was not found.
Symbol '000240' was not found.
Symbol '024720' was not found.
Symbol '005440' was not found.
Symbol '378850' was not found.


# initialize 살펴보기

In [15]:
def initialize(context):
    pass

## Slippage 설정

In [16]:
from zipline.api import set_slippage

In [17]:
def initialize(context):
    set_slippage()

### Slippage 종류 설정

In [19]:
from zipline.finance import slippage

#### FixedSlippage
- slippage_price = close +- (spread / 2)

In [22]:
def initialize(context):
    set_slippage(slippage.FixedSlippage(spread=0.1))

#### VolumeShareSlippage

In [23]:
def initialize(context):
    set_slippage(slippage.VolumeShareSlippage(volume_limit=0.2, price_impact=0.1))

## 자산군 설정

In [24]:
def initialize(context):
    context.universe = assets

## 거래 수수료 설정

In [25]:
from zipline.api import set_commission

In [26]:
def initialize(context):
    set_commission()

### 수수료 종류 설정

In [27]:
from zipline.finance import commission

#### PerShare
한주당 비용을 가지고 거래 수수료를 계산

In [29]:
def initialize(context):
    set_commission(commission.PerShare(cost=0.001, min_trade_cost=0.0))

#### PerTrade
한번 거래당 비용으로 계산 

In [30]:
def initialize(context):
    set_commission(commission.PerTrade(cost=0.02))

#### PerDollar
거래당 고정된 비용이 나감

In [31]:
def initialize(context):
    set_commission(commission.PerDollar(cost=0.03))

## BenchMark 설정
- Parameter
[zipline.assets.Asset](https://zipline.ml4trading.io/api-reference.html#:~:text=benchmark%20(zipline.assets.Asset)%20%E2%80%93%20The%20asset%20to%20set%20as%20the%20new%20benchmark.) – The asset to set as the new benchmark.


- Asset 객처로 benchmark 데이터를 넣어준다.

In [32]:
from zipline.api import set_benchmark

In [33]:
def initialize(context):
    set_benchmark()

## Long 포트폴리오로만 설정

In [34]:
from zipline.api import set_long_only

In [35]:
def initialize(context):
    set_long_only()

## Max Leverage 설정

In [36]:
from zipline.api import set_max_leverage

In [37]:
def initialize(context):
    set_max_leverage(max_leverage=2.1)

## Scheduling Function 설정 (중요!!)
- reblancing할때 많이 사용

In [38]:
from zipline.api import schedule_function

In [39]:
def initialize(context):
    schedule_function(func='', date_rule='', time_rule='', half_days='', calendar='')

#### date_rules 종류 보기

In [40]:
from zipline.api import date_rules

In [41]:
date_rules.every_day()

date_rules.month_end()

date_rules.month_start()

date_rules.week_end()

date_rules.week_start()


##### time_rules 설정

In [42]:
from zipline.api import time_rules

In [43]:
time_rules.market_open()

time_rules.market_close()

# handle_data 설정

In [44]:
def handle_data(context, data):
    pass 

## can_trade() 거래가 가능한지 확인

In [45]:
def handle_data(context, data):
    data.can_trade(asset)

## current 백테스팅 거래시점에서 데이터 

In [47]:
def handle_data(context, data):
    data.current(assts='', fields='')

Parameters
- assets (zipline.assets.Asset or iterable of zipline.assets.Asset) 
    + The asset(s) for which data is requested.

- fields (str or iterable[str]) 
     + Requested data field(s). Valid field names are: “price”, “last_traded”, “open”, “high”, “low”, “close”, and “volume”.

## history 백테스팅 시점에서 과거 데이터

In [48]:
def handle_data(context, data):
    data.history(assts='', fields='', bar_counts=10, frequency='')

Parameters
- assets (zipline.assets.Asset or iterable of zipline.assets.Asset) 
     + The asset(s) for which data is requested.

- fields (string or iterable of string.) 
     + Requested data field(s). Valid field names are: “price”, “last_traded”, “open”, “high”, “low”, “close”, and “volume”.

- bar_count (int) 
     + Number of data observations requested.

- frequency (str) 
     + String indicating whether to load daily or minutely data observations. Pass ‘1m’ for minutely data, ‘1d’ for daily data

# Order 주문 관련 함수 

## order
고정된 수의 주식을 거래

In [50]:
from zipline.api import order

def handle_data(context, data):
    order(asset=None , amount=None , limit_price=None, stop_price=None, style=None)

## order_value
고정된 금액을 구매

In [51]:
from zipline.api import order_value

def handle_data(context, data):
    order_value(asset=None, value=None, limit_price=None, stop_price=None, style=None)

## order_percent
포트폴리오에 해당된 비율만큼 거래

In [52]:
from zipline.api import order_percent

def handle_data(context, data):
    order_percent(asset=None, percent=None, limit_price=None, stop_price=None, style=None)

## order_target
목표 주식 수에 대한 포지션을 조정한다. 아직 존재하지 않은 경우 새주문, 포지션이 존재하면 목표 주식수 차이만큼 다시 주문

In [53]:
from zipline.api import order_target

def handle_data(context, data):
    order_target(asset=None, target=None, limit_price=None, stop_price=None, style=None)

## order_target_value
포지션을 목표 값으로 조정하기 위해 주문하십시오. 위치가 아직 존재하지 않는 경우 이는 새로 주문하는 것과 같습니다. 포지션이 존재한다면 목표 값과 현재 값의 차이를 주문하는 것과 같습니다.



In [55]:
from zipline.api import order_target_value

def handle_data(context, data):
    order_target_value(asset=None, target=None, limit_price=None, stop_price=None, style=None)

## order_target_percent
현재 포트폴리오의 목표 비율로 포지션을 조정하는 주문을하십시오. 포지션이 아직 존재하지 않는 경우 이는 새로 주문하는 것과 같습니다. 포지션이 존재한다면 목표 퍼센트와 현재 퍼센트 사이의 차이를 주문하는 것과 같습니다.

In [56]:
from zipline.api import order_target_percent

def handle_data(context, data):
    order_target_percent(asset=None, target=None, limit_price=None, stop_price=None, style=None)

### style parameter에서 원하는 주문 방식을 정할 수 있다.

In [60]:
from zipline.finance.execution import ExecutionStyle
def handle_data(context, data):
    order_target_percent(asset=None, 
                         target=None, 
                         limit_price=None, 
                         stop_price=None, 
                         style=ExecutionStyle.get_limit_price()
                        )
#     종류
#     ExecutionStyle.get_limit_price()
#     ExecutionStyle.get_stop_price()
